In [1]:
import torch
import torch.nn as nn

In [2]:
embed_size = 128
hidden_size = 512

In [3]:
lstm = nn.LSTM(embed_size, hidden_size, batch_first=True, bidirectional=True)

In [4]:
x = torch.rand(1, 2000, 128)

In [11]:
output, (hn, cn) = lstm(x)

In [12]:
hn.size()

torch.Size([2, 1, 512])

In [33]:
output.size()

torch.Size([1, 2000, 1024])

In [27]:
hn[0][0]

tensor([ 0.0890,  0.0503, -0.1258,  0.0348,  0.0773,  0.0296,  0.0683, -0.1334,
        -0.0785,  0.1435,  0.1165,  0.0447,  0.0304, -0.0194, -0.0036, -0.1340,
         0.0818,  0.0420, -0.0170,  0.0313,  0.0711, -0.0999, -0.0478, -0.0976,
         0.0059, -0.2388, -0.0579, -0.0516, -0.0657, -0.0896, -0.0562,  0.0019,
        -0.0259, -0.0272,  0.1039,  0.0165, -0.0452, -0.0452, -0.0048,  0.1142,
         0.0230,  0.0059, -0.0060, -0.0641, -0.0324,  0.1596, -0.0969,  0.0614,
         0.1177, -0.0420,  0.1390, -0.0454, -0.0312, -0.0731,  0.0407, -0.0226,
         0.0078,  0.0410, -0.1303,  0.0246,  0.0103, -0.1485, -0.0549, -0.0267,
         0.0243,  0.0417,  0.0198,  0.0057, -0.0479,  0.0122, -0.0588, -0.0612,
        -0.1731,  0.0549,  0.0408,  0.0732, -0.0445, -0.0473, -0.1210, -0.0847,
         0.0129,  0.1125, -0.0053,  0.0241, -0.1423, -0.0316,  0.0709, -0.0529,
         0.0664, -0.1628,  0.0511,  0.0147,  0.1425,  0.0811, -0.0684,  0.0026,
        -0.1069, -0.0261,  0.0207, -0.06

In [30]:
output[0][-1][:10]

tensor([ 0.0890,  0.0503, -0.1258,  0.0348,  0.0773,  0.0296,  0.0683, -0.1334,
        -0.0785,  0.1435], grad_fn=<SliceBackward>)

In [31]:
hn.size()

torch.Size([2, 1, 512])

In [32]:
cn.size()

torch.Size([2, 1, 512])

In [35]:
lstm2 = nn.LSTM(embed_size, hidden_size, batch_first=True)

In [36]:
out, (h, c) = lstm2(x)

In [37]:
h.size()

torch.Size([1, 1, 512])

In [38]:
h

tensor([[[ 5.0007e-02,  1.1504e-01, -8.0164e-03, -2.8628e-02,  4.3624e-02,
          -4.8098e-02, -8.9852e-02, -2.0751e-02, -1.4995e-02,  2.1779e-02,
           1.4510e-01,  3.7402e-02, -2.4306e-02,  8.3935e-02,  1.3019e-01,
           5.2696e-02,  5.8331e-03,  2.3580e-02,  5.0516e-03, -1.6813e-01,
           2.0978e-02,  7.0036e-02,  2.5464e-02, -8.2757e-03, -8.2930e-02,
          -2.4754e-01,  3.6266e-02, -8.9510e-02,  1.0058e-01, -3.9296e-02,
          -3.2925e-02, -3.0055e-02, -1.1519e-01,  1.6550e-02, -1.0098e-01,
           2.4721e-02, -4.6358e-02,  1.4646e-01,  6.0281e-02, -3.3340e-02,
          -1.5491e-01, -1.3073e-01, -1.1565e-01,  4.8550e-02, -5.9586e-02,
          -1.9076e-01, -1.0272e-01, -8.1003e-02,  7.1642e-02, -7.8382e-02,
          -6.5359e-02,  1.9230e-03,  1.1971e-01,  1.7767e-01,  3.5265e-02,
           4.9145e-02, -9.9055e-02,  3.6508e-02,  1.4252e-01, -1.0448e-01,
           2.6643e-02, -1.9212e-02,  2.4304e-02, -1.3443e-01, -9.2144e-02,
           5.3559e-02, -4

In [39]:
c[-1].size()

torch.Size([1, 512])

In [45]:
x = torch.randn(2, 1, 1)#为1可以扩展为3和3 
x

tensor([[[ 0.2492]],

        [[-0.5608]]])

In [46]:
y = torch.randn(2, 3, 3)

In [47]:
y

tensor([[[-1.3034,  0.2991, -0.4937],
         [-0.8327,  1.0045,  0.2131],
         [-0.8213,  0.3709, -0.2830]],

        [[ 0.8204, -2.0705,  0.1645],
         [ 1.8283,  0.9639, -0.8203],
         [-0.1718,  0.4644, -1.1035]]])

In [43]:
x = x.expand_as(y)
x

tensor([[[-2.2631, -2.2631, -2.2631],
         [-2.2631, -2.2631, -2.2631],
         [-2.2631, -2.2631, -2.2631]],

        [[ 1.2091,  1.2091,  1.2091],
         [ 1.2091,  1.2091,  1.2091],
         [ 1.2091,  1.2091,  1.2091]]])

In [48]:
z = x.expand_as(y).contiguous()

In [49]:
z

tensor([[[ 0.2492,  0.2492,  0.2492],
         [ 0.2492,  0.2492,  0.2492],
         [ 0.2492,  0.2492,  0.2492]],

        [[-0.5608, -0.5608, -0.5608],
         [-0.5608, -0.5608, -0.5608],
         [-0.5608, -0.5608, -0.5608]]])

In [51]:
z.size()

torch.Size([2, 3, 3])